# sample2

We run [SPARK-X](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02404-0) on the sample.

Here are the libraries we need.

In [1]:
library(SPARK)
library(Seurat)
library(dplyr)
library(future)
library(readr)
library(tibble)

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




We create the output directory for this noteboook.
Every outputs will save there.

In [2]:
dir.create("output", showWarnings=F)

## Create a reference object

We load the digital expression matrix and the spatial information of the sample.

In [3]:
counts <- Seurat::Read10X("sample2", gene.column=2)

spatial <-
    readr::read_csv("sample2.csv") %>%
    tibble::column_to_rownames("Barcode")

spatial <- spatial[colnames(counts),]

Rows: 58092 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Barcode
dbl (2): x, y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


We remove mitochondrial genes if present.

In [4]:
mt_idx <- grep("mt-", rownames(counts), ignore.case=T)
if ( length(mt_idx) != 0 )
{
   counts <- counts[-mt_idx,]
}

We run SPARK-X.

In [5]:
spark <- 
   SPARK::sparkx(
      count_in=counts,
      locus_in=as.matrix(spatial),
      numCores=12,
      option="mixture",
      verbose=T
   )

## ===== SPARK-X INPUT INFORMATION ==== 
## number of total samples: 42215 
## number of total genes: 25805 
## Running with 12 cores 
## Testing With Projection Kernel
## Testing With Gaussian Kernel 1
## Testing With Gaussian Kernel 2
## Testing With Gaussian Kernel 3
## Testing With Gaussian Kernel 4
## Testing With Gaussian Kernel 5
## Testing With Cosine Kernel 1
## Testing With Cosine Kernel 2
## Testing With Cosine Kernel 3
## Testing With Cosine Kernel 4
## Testing With Cosine Kernel 5


Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”
Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”
Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”
Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”
Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”
Warning message in FUN(newX[, i], ...):
“There are p-values that are exactly 1!”


We export the results.

In [6]:
spark$res_mtest %>%
   tibble::rownames_to_column("gene") %>%
   readr::write_csv("output/results.csv")

## Session info

In [7]:
sessionInfo()

R version 4.2.1 (2022-06-23)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu Kinetic Kudu (development branch)

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tibble_3.1.7       readr_2.1.2        future_1.26.1      dplyr_1.0.9       
[5] SeuratObject_4.0.4 Seurat_4.1.0       SPARK_1.1.1       

loaded via a namespace (and not attached):
  [1] Rtsne_0.16            colorspace_2.0-3      deldir_1.0-6         
  [4] ellipsis_0.3.2        ggridges_0.5.3        IRdisplay_